# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
import os
from openai import OpenAI

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), base_url='https://dashscope.aliyuncs.com/compatible-mode/v1')

def get_completion(prompt: str, system_prompt=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## Lesson

We are using the OpenAI-compatible API format to interact with Qwen. The [Chat Completions API](https://platform.openai.com/docs/guides/chat) allows us to send messages to Qwen and receive responses.

At minimum, a call to Qwen using the Chat Completions API requires the following parameters:
- `model`: the model name (we're using `qwen-plus`)

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Qwen may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Qwen to stop generating mid-word or mid-sentence.

- `messages`: an array of input messages. The models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new message, you specify the prior conversational turns with the messages parameter, and the model then generates the next message in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). The first message must always use the user `role`.

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Qwen's response. For these lessons and exercises, we have set `temperature` to 0.

For a complete list of all API parameters, visit the [OpenAI API documentation](https://platform.openai.com/docs/api-reference/chat).

### Examples

Let's take a look at how Qwen responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Qwen's response will appear below the block.

In [ ]:
# Prompt
PROMPT = "Hi Qwen, how are you?"

# Print Qwen's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Qwen's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Qwen's response
print(get_completion(PROMPT))

Now let's take a look at some prompts that do not include the correct Chat Completions API formatting. For these malformatted prompts, the API returns an error.

First, we have an example of an API call that lacks `role` and `content` fields in the `messages` array.

In [ ]:
# Get Qwen's response
response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Qwen, how are you?"}
        ]
    )

# Print Qwen's response
print(response.choices[0].message.content)

Here's a prompt that fails to alternate between the `user` and `assistant` roles.

In [ ]:
# Get Qwen's response
response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Qwen's response
print(response.choices[0].message.content)

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Qwen to continue from where you left off (more on that in later chapters).

#### System Prompts

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Qwen** before presenting it with a question or task in the "User" turn. 

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Setup](#setup) section of the notebook). 

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Example

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Qwen's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Why use a system prompt? A **well-written system prompt can improve Qwen's performance** in a variety of ways, such as increasing Qwen's ability to follow rules and instructions.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get Qwen to **count to three.** The output will also indicate whether your solution is correct.

In [ ]:
import re
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Qwen's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Qwen's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Qwen respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Qwen's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Qwen's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Hi Qwen, how are you?"

# Print Qwen's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Qwen's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Qwen's response
print(get_completion(PROMPT))

In [ ]:
# Get Qwen's response
response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Qwen, how are you?"}
        ]
    )

# Print Qwen's response
print(response.choices[0].message.content)

In [ ]:
# Get Qwen's response
response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Qwen's response
print(response.choices[0].message.content)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Qwen's response
print(get_completion(PROMPT, SYSTEM_PROMPT))